In [1]:
"""
Classify titles based on word commonality and string content
"""

import sqlite3
import re
import pandas as pd
import numpy as np
import nltk
import sklearn
from sklearn import model_selection, svm, neighbors, preprocessing
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
import pickle
from pathlib import Path
nltk.download('stopwords')
home = str(Path.home())
print(home)
# Variable fields to adapt to different data sets --------------------------------------------------------------
# Rank your role-levels in this dictionary
values_to_replace = {'clevel': 1, 'vplevel': 2, 'directorlevel': 3, 'managerlevel': 4, 'staff': 5}
# Level field in your data
level_field = 'Management Level'
# Title Field in your data
title_field = 'Title'

# st.stem() sklearn's word stemming algorithm
st = nltk.stem.snowball.SnowballStemmer("english", ignore_stopwords=True)
# sklearn's vectorizor for bag of words analysis
count_vect = sklearn.feature_extraction.text.CountVectorizer()
TfidfVectorizer = sklearn.feature_extraction.text.TfidfVectorizer()
# sklearn's normalizer to convert to zero-mean
normalizer = preprocessing.Normalizer(copy=True, norm='l2')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Greg\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\Greg


In [2]:
# Import Data / Process Functions-----------------------------------------------------------------------------------------

# SQL Connection to Source File
db = sqlite3.connect(
    '{}\\directories\\database.db'.format(home))
contacts2 = pd.read_sql("""SELECT ContactID, C_EmailAddress,
                        C_Title, C_Management_Level FROM contacts;""", con=db)
db.close()

contacts = pd.read_excel('{}\\OneDrive - Softchoice\\Documents\\PyCharmProjects\\Contact-Ranking\\Training_All_Conacts - Unique.xlsx'.format(home))
contacts.fillna(inplace=True, value="")


# Training Data Set, if you want to use separate training data
def import_training_data():

    df = pd.read_csv(
        '{}\\OneDrive - Softchoice\\Documents\\PyCharmProjects\\Contact-Ranking\\Training_Export2.csv'.format(home))
    # df = df[~df['Management Level'].str.contains("Senior-level manager")]
    df = df[['Title', 'Management Level', 'TitleFormatted']]
    df.rename(index=str, columns={'Title': title_field, 'Management Level': level_field}, inplace=True)

    return df


# Import Title Abbreviations/replacements to improve accuracy
title_abrev = pd.read_excel(
    '{}\\OneDrive - Softchoice\\Documents\\PyCharmProjects\\Contact-Ranking\\title_abrev.xlsx'.format(home), sep=',')
title_abrev.Title = title_abrev.Title.str.split(',').str[0].str.lower()
title_abrev.to_pickle('title_abrev.p')


def replace_str(series):
    """
    A short function to replace any abbreviations with their full form
    :param series: series to be cleansed
    :return: the cleansed series
    """
    print(series.tail())
    for a, t in zip(title_abrev.Abrev, title_abrev.Title):
        pattern = re.compile(r"\b{}\b".format(re.escape(a)), flags=re.IGNORECASE)
#         print(a, t)
        series = series.apply(lambda x: re.sub(pattern, t, x))
    print('Replaced')
    print(series.tail())
    return series


def process(df):
    """
    The main processing stem for data inputs
    :param df: data frame to be cleansed
    :return: cleansed data frame
    """
    df[level_field] = df[level_field].str.lower().str.replace('[^\w\s]', '')
    df[level_field] = df[level_field].map(values_to_replace)
    df['TitleFormatted'] = df[title_field].str.lower().str.replace('[^\w\s]', ' ').str.replace('\s+', ' ')
    df['TitleFormatted'] = replace_str(df['TitleFormatted'])  # Use function to replace abbreviations and misspellings
    df['TitleFormatted'] = df['TitleFormatted'].apply(
        lambda row: ' '.join([st.stem(y) for y in row.split(" ")]))

    return df


In [3]:
# Process Data --------------------------------------------------------------------------------------------------

print("Processing contact data.")
contacts = process(contacts)
# print(contacts[level_field])


Processing contact data.
58882     ceo and director
58883     cio and director
58884      cto and directo
58885    ciso and director
58886     cno and director
Name: TitleFormatted, dtype: object
Replaced
58882               chief executive officer and director
58883             chief information officer and director
58884              chief technology officer and director
58885    chief information security officer and director
58886                 chief nursing officer and director
Name: TitleFormatted, dtype: object


In [4]:
# rainking_data = import_training_data()

# Hot Swap Training Data Set
# training_contacts = rainking_data

In [5]:
# ---------------------------------------------------------------------------------------------------------------
# Split data set
contacts[level_field].replace('', np.nan, inplace=True)
data_set = contacts[pd.notnull(contacts[level_field])]

print("Transforming titles to vectors.")
X = TfidfVectorizer.fit_transform(data_set['TitleFormatted'].values)
y = data_set[level_field]

f = open(
        '{}\\Directory\\Title_Vectorizor.pickle'.format(home), 'wb')
pickle.dump(TfidfVectorizer, f)
f.close()

# Normalize the data set to optimise for Linear SVC
X = normalizer.transform(X)

Transforming titles to vectors.


In [6]:
stem = "ex"

# print(data_set.head()) # Describe the data_set
print(X.shape)
print("Count of {word}: {count}".format(count=str(TfidfVectorizer.vocabulary_.get(u'{}'.format(stem))), word=stem))
# print(y.shape)
# print(bag_of_words.vocabulary_.get('man')) # Print how many times a particular stem appears


(58887, 8537)
Count of ex: 3002


In [7]:
# Train Classifier --------------------------------------------------------------------------------------------------------

# Options: M = Multinomial, B = Bernoulli, SVM = SVM
MN = 'SVM'

# Splits the data into training and testing data sets (20%)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.1)

if MN == 'B':
    clf = BernoulliNB()
elif MN == 'SVM':
#     clf = svm.LinearSVC()
    clf = sklearn.linear_model.SGDClassifier(n_jobs=-1, alpha=1e-3)
else:
    clf = MultinomialNB()
print("Training classifier.")
clf.fit(X, y)

if MN == 'B':
    f = open(
        '{}\\Directory\\Title_Classifier_B.pickle'.format(home), 'wb')
elif MN == 'SVM':
    f = open(
        '{}\\Directory\\Title_Classifier_SVM.pickle'.format(home), 'wb')
else:
    f = open(
        '{}\\Directory\\Title_Classifier_MN.pickle'.format(home), 'wb')
pickle.dump(clf, f)
f.close()

# Return Accuracy score
accuracy = clf.score(X_test, y_test)
print("Estimated accuracy is: {}%".format(accuracy))

Training classifier.
Estimated accuracy is: 0.8970962812022415%


C:\Users\Greg\AppData\Roaming\Python\Python36\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [8]:
# Analyse -------------------------------------------------------------------------------------------------------------
# Create a copy of original data set to analyze
contacts2.rename(index=str, columns={'C_Management_Level1': 'Management Level','C_Title': 'Title'}, inplace=True)
contacts2 = process(contacts2)
X_process = TfidfVectorizer.transform(contacts2['TitleFormatted'])
print("Predicting management level.")
prediction = clf.predict(X_process)
contacts2[level_field+'_P'] = prediction
print("Finished Predicting.")

# Create a column that highlights where the prediction is different than the current value ----------------------------
print("Adding column to highlight differences in contact table.")
contacts2["Is_Same"] = np.where(contacts2[level_field] == contacts2[level_field+'_P'], 'yes', 'no')
# ---------------------------------------------------------------------------------------------------------------------

# drop working column and export to csv
contacts2.drop('TitleFormatted', axis=1, inplace=True)
print(contacts2.describe())
print("Exporting predictions.")
contacts2.to_csv('test_predictions.csv', sep='|')

217407               
217408               
217409               
217410               
217411    it director
Name: TitleFormatted, dtype: object
Replaced
217407                                   
217408                                   
217409                                   
217410                                   
217411    information technology director
Name: TitleFormatted, dtype: object
Predicting management level.
Finished Predicting.
Adding column to highlight differences in contact table.
       Management Level  Management Level_P
count     112640.000000       217412.000000
mean           3.534570            3.954671
std            1.353595            1.310041
min            1.000000            1.000000
25%            3.000000            3.000000
50%            4.000000            5.000000
75%            5.000000            5.000000
max            5.000000            5.000000
Exporting predictions.
